In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Ticker list
nifty50_tickers = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'ICICIBANK.NS',
    'INFY.NS', 'ITC.NS', 'KOTAKBANK.NS', 'BHARTIARTL.NS',
    'LT.NS', 'SBIN.NS', 'BAJFINANCE.NS', 'ASIANPAINT.NS',
    'HCLTECH.NS', 'MARUTI.NS', 'TITAN.NS', 'SUNPHARMA.NS',
    'AXISBANK.NS', 'NTPC.NS', 'ONGC.NS', 'NESTLEIND.NS',
    'INDUSINDBK.NS', 'POWERGRID.NS', 'M&M.NS', 'ULTRACEMCO.NS',
    'TECHM.NS', 'WIPRO.NS', 'GRASIM.NS', 'ADANIPORTS.NS',
    'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BRITANNIA.NS', 'CIPLA.NS',
    'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS',
    'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'JSWSTEEL.NS',
    'SBILIFE.NS', 'SHREECEM.NS', 'TATASTEEL.NS', 'UPL.NS'
]

start_date = '2022-01-01'
end_date = '2024-09-01'



data = yf.download(nifty50_tickers, start=start_date, end=end_date, auto_adjust=True)['Close']


valid_tickers = data.columns[data.notna().sum() > len(data) * 0.9]
data = data[valid_tickers]

if data.empty:
    raise ValueError("No usable adjusted close data found after filtering. Try changing date range or ticker list.")

print(f"\nData downloaded for {len(data.columns)} valid tickers:")
print(list(data.columns))

# Calculate returns and volatility
returns = data.pct_change().dropna()
rolling_vol = returns.rolling(window=20).std()

volatility_results = pd.DataFrame(index=data.columns, columns=['Actual_Vol', 'RollingMean_Forecast', 'EWMA_Forecast', 'AR1_Forecast'])

# Forecast loop
for stock in data.columns:
    stock_returns = returns[stock].dropna()
    stock_vol = rolling_vol[stock].dropna()

    if len(stock_vol) < 30:
        continue

    try:
        # Rolling Mean Forecast
        vol_forecast_rm = stock_vol.rolling(window=5).mean()
        volatility_results.loc[stock, 'RollingMean_Forecast'] = vol_forecast_rm.iloc[-1]

        # EWMA
        lambda_ = 0.94
        vol_squared = stock_returns ** 2
        ewma_vol = vol_squared.ewm(alpha=1 - lambda_).mean() ** 0.5
        volatility_results.loc[stock, 'EWMA_Forecast'] = ewma_vol.iloc[-1]

        # AR(1) Forecast
        model = AutoReg(stock_vol, lags=1).fit()
        ar_forecast = model.predict(start=len(stock_vol), end=len(stock_vol))
        volatility_results.loc[stock, 'AR1_Forecast'] = ar_forecast.iloc[0]

        # Actual Volatility
        volatility_results.loc[stock, 'Actual_Vol'] = stock_vol.iloc[-1]

    except Exception as e:
        print(f"Skipped {stock} due to error: {e}")

# Drop incomplete rows
volatility_results = volatility_results.dropna()

if volatility_results.empty:
    raise ValueError("No valid volatility forecasts after processing.")

# RMSE Calculation
def calculate_rmse(actual, forecast):
    actual = pd.to_numeric(actual, errors='coerce')
    forecast = pd.to_numeric(forecast, errors='coerce')
    valid = actual.notna() & forecast.notna()
    return np.sqrt(mean_squared_error(actual[valid], forecast[valid]))

rmse_scores = {
    'RollingMean': calculate_rmse(volatility_results['Actual_Vol'], volatility_results['RollingMean_Forecast']),
    'EWMA': calculate_rmse(volatility_results['Actual_Vol'], volatility_results['EWMA_Forecast']),
    'AR1': calculate_rmse(volatility_results['Actual_Vol'], volatility_results['AR1_Forecast'])
}

best_method = min(rmse_scores, key=rmse_scores.get)
print(f"\nBest method: {best_method} (RMSE: {rmse_scores[best_method]:.4f})")

# Normalize best forecast
volatility_results['Best_Forecast'] = volatility_results[f"{best_method}_Forecast"]
volatility_results['Normalized_Vol'] = (
    (volatility_results['Best_Forecast'] - volatility_results['Best_Forecast'].min()) /
    (volatility_results['Best_Forecast'].max() - volatility_results['Best_Forecast'].min())
)

# Show top 5
print("\nTop 5 stocks by forecasted volatility:")
print(volatility_results.sort_values('Best_Forecast', ascending=False).head(5))

# Save
volatility_results.to_csv('volatility_forecasts.csv')
print("\nResults saved to 'volatility_forecasts.csv'")


[**********************95%*********************  ]  42 of 44 completedERROR:yfinance:Failed to get ticker 'BAJAJ-AUTO.NS' reason: Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  44 of 44 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BAJAJ-AUTO.NS']: HTTPError('HTTP Error 401: ')



Data downloaded for 43 valid tickers:
['ADANIPORTS.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJFINSV.NS', 'BAJFINANCE.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'ICICIBANK.NS', 'INDUSINDBK.NS', 'INFY.NS', 'ITC.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS', 'SUNPHARMA.NS', 'TATASTEEL.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'UPL.NS', 'WIPRO.NS']

Best method: AR1 (RMSE: 0.0001)

Top 5 stocks by forecasted volatility:
             Actual_Vol RollingMean_Forecast EWMA_Forecast AR1_Forecast  \
Ticker                                                                    
ONGC.NS        0.028118              0.02851      0.023583     0.027938   
HINDALCO.NS    0.022832             0.022878      0.0

In [ ]:
risk_averse = 0.5 #Can be anything from 0 to 1
volatility_results['dist'] = abs(volatility_results['Normalized_Vol'] - risk_averse).astype(float)

mindist = volatility_results.nsmallest(3,'dist')
print(mindist)


              Actual_Vol RollingMean_Forecast EWMA_Forecast AR1_Forecast  \
Ticker                                                                     
HDFCLIFE.NS     0.017701             0.018103      0.016007     0.017667   
TECHM.NS        0.018211             0.018203      0.015777     0.018144   
ADANIPORTS.NS   0.018129             0.018602      0.017386     0.018263   

              Best_Forecast Normalized_Vol      dist  
Ticker                                                
HDFCLIFE.NS        0.017667       0.490273  0.009727  
TECHM.NS           0.018144       0.513945  0.013945  
ADANIPORTS.NS      0.018263       0.519834  0.019834  


## So best suited stocks for risk_averse = 0.5 are HDCFLIFE, TECHM, UPL

In [ ]:
capital = 10_000_000 # 1/3rd capital for each stock. Equal Weightage

def backtest_strategy(data, capital=10_000_000/3, short_window=10, long_window=30, risk_free_rate=0.05):
    df = data.copy()
    df['short_ma'] = df['Close'].rolling(short_window).mean()
    df['long_ma'] = df['Close'].rolling(long_window).mean()

    df['signal'] = 0
    df.loc[df['short_ma'] > df['long_ma'], 'signal'] = 1
    df.loc[df['short_ma'] < df['long_ma'], 'signal'] = -1
    df['position'] = df['signal'].shift()

    df['returns'] = df['Close'].pct_change()
    df['strategy_returns'] = df['returns'] * df['position']

    df['capital'] = capital * (1 + df['strategy_returns']).cumprod()
    df['pnl'] = df['capital'].diff()

    cumulative_return = df['capital'].iloc[-1] / capital - 1
    sharpe_ratio = df['strategy_returns'].mean() / df['strategy_returns'].std() * np.sqrt(252)
    max_drawdown = (df['capital'] / df['capital'].cummax() - 1).min()

    metrics = {
        'Initial Capital': capital,
        'Final Capital': df['capital'].iloc[-1],
        'Cumulative Return': cumulative_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
    }

    signals = df[['Close', 'short_ma', 'long_ma', 'signal', 'position', 'capital', 'pnl', 'strategy_returns']]
    return {'signals': signals, 'metrics': metrics}

#BackTesting ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\

tickers = {
    'UPL': 'UPL.NS',
    'TECHM': 'TECHM.NS',
    'HDFCLIFE': 'HDFCLIFE.NS'
}
results = {}

for name, symbol in tickers.items():
    df = yf.download(symbol, start=start_date, end='2025-01-01', auto_adjust=True)
    df = df[['Close']]
    result = backtest_strategy(df)
    results[name] = result['metrics']

# Conclusion
conclusion_df = pd.DataFrame(results).T.round(2)  #
print()
print(conclusion_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


          Initial Capital  Final Capital  Cumulative Return  Sharpe Ratio  \
UPL            3333333.33     3665247.12               0.10          0.25   
TECHM          3333333.33     3872643.60               0.16          0.33   
HDFCLIFE       3333333.33     2954981.52              -0.11         -0.03   

          Max Drawdown  
UPL              -0.29  
TECHM            -0.35  
HDFCLIFE         -0.50  
